In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [2]:
df = pd.read_csv("clean_scout.csv")
df.isnull().sum()

make_model                 0
body_type                 60
price                      0
vat                     4513
km                      1024
type                       2
previous_owners         6640
inspection_new         11987
make                       0
model                      0
body_color               597
paint_type              5772
nr_of_doors              212
nr_of_seats              977
gearing_type               0
cylinders               5680
drive_chain             6858
fuel                       0
co2_emission            2436
emission_class          3628
comfort_convenience      920
entertainment_media     1374
extras                  2962
safety_security          982
gears                   4712
age                     1597
hp_kW                     88
upholstery_type        14617
warranty_months        11066
displacement_cc          496
weight_kg               6974
cons_comb               2033
cons_city               2436
cons_country            2376
dtype: int64

In [3]:
df.make_model.value_counts(dropna=False)

Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: make_model, dtype: int64

## body_type

In [4]:
df.body_type.value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [5]:
df.body_type.fillna("Other",inplace=True)

In [6]:
df.groupby("body_type").nr_of_doors.describe()

,count,mean,std,min,25%,50%,75%,max
body_type,,,,,,,,
Compact,3115.000,4.361,0.901,1.000,4.000,5.000,5.000,5.000
Convertible,8.000,2.375,1.061,2.000,2.000,2.000,2.000,5.000
Coupe,24.000,3.375,0.824,2.000,3.000,3.000,3.250,5.000
Off-Road,55.000,4.982,0.135,4.000,5.000,5.000,5.000,5.000
Other,345.000,4.858,0.476,2.000,5.000,5.000,5.000,5.000
Sedans,7782.000,4.705,0.585,2.000,5.000,5.000,5.000,5.000
Station wagon,3519.000,4.742,0.441,2.000,4.000,5.000,5.000,5.000
Transporter,87.000,4.897,0.432,3.000,5.000,5.000,5.000,5.000
Van,772.000,4.886,0.330,4.000,5.000,5.000,5.000,7.000


In [7]:
df.body_type.apply("mode")

0    Sedans
dtype: object

In [8]:
df.replace("Other","Sedans",inplace=True)

## vat

In [9]:
df.vat.value_counts(dropna=False)  # here nan could be non-negotiable. but we cannot be sure. so i will just drop the column

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

In [10]:
df.drop("vat", axis=1, inplace=True)

## km

km can be related to age and previous_owners

In [11]:
df[df.km.isnull()].age.value_counts(dropna=False)

NaN      838
0.000    147
1.000     38
2.000      1
Name: age, dtype: int64

In [12]:
df[df.km.isnull()].previous_owners.value_counts(dropna=False)

NaN      887
0.000     80
1.000     57
Name: previous_owners, dtype: int64

In [13]:
df.groupby(["age","previous_owners"]).km.describe()

count      mean       std       min       25%  \
age   previous_owners                                                    
0.000 0.000              2.000    10.000     0.000    10.000    10.000   
      1.000           1535.000  2140.048  4097.526     1.000    10.000   
      2.000             10.000  8865.000 23817.262    10.000    15.000   
1.000 0.000              8.000 27179.375  8253.035 15000.000 21807.750   
      1.000           2952.000 18026.141 10642.585     1.000  9941.500   
      2.000             75.000 16729.173  9910.780    50.000  9750.000   
2.000 0.000              6.000 56883.667 23572.241 19300.000 45500.000   
      1.000           1710.000 42927.127 30890.123     1.000 21061.250   
      2.000            296.000 30664.311 18980.036    15.000 17768.000   
      3.000              7.000 39957.857 19237.062 17390.000 25595.000   
      4.000              1.000 19700.000       NaN 19700.000 19700.000   
3.000 0.000             10.000 60492.900 20985.402 23580.000 52530.250   
      1.000           2026.000 83261.406 41472.994    10.000 52099.000   
      2.000            397.000 56134.713 31651.648  1800.000 36934.000   
      3.000             10.000 54629.900 25500.682 27600.000 40500.000   
      4.000              1.000 57000.000       NaN 57000.000 57000.000   

                            50%        75%        max  
age   previous_owners                                  
0.000 0.000              10.000     10.000     10.000  
      1.000              51.000   3275.500  88000.000  
      2.000             297.500   2500.000  76300.000  
1.000 0.000           28800.000  32750.000  37592.000  
      1.000           18000.000  25304.250 113392.000  
      2.000           15455.000  25066.000  47000.000  
2.000 0.000           64000.000  67501.500  86000.000  
      1.000           33921.500  58814.250 317000.000  
      2.000           23222.500  45050.000 116184.000  
      3.000           37125.000  54000.000  66000.000  
      4.000           19700.000  19700.000  19700.000  
3.000 0.000           62000.000  64875.000  91883.000  
      1.000           80024.500 110000.000 291800.000  
      2.000           49351.000  68550.000 229500.000  
      3.000           50750.000  62524.250 116000.000  
      4.000           57000.000  57000.000  57000.000

In [14]:
df.km.fillna(df.groupby(["age","previous_owners"]).km.transform("mean"),inplace=True)

In [15]:
df.km.value_counts(dropna=False)

10.000       1045
NaN           974
1.000         367
5.000         170
50.000        148
             ... 
20719.000       1
91910.000       1
39860.000       1
28084.000       1
57889.000       1
Name: km, Length: 6693, dtype: int64

In [16]:
# there are still nan values. we can look at age column

 ## age

In [17]:
df.age.value_counts(dropna=False)

1.000    4522
3.000    3674
2.000    3273
0.000    2853
NaN      1597
Name: age, dtype: int64

In [18]:
df[df.age.isnull()].previous_owners.value_counts()

0.000    162
1.000     21
Name: previous_owners, dtype: int64

In [19]:
df.groupby("age").km.describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.000,2740.000,2086.034,5332.476,1.000,10.000,50.000,2998.000,127022.000
1.000,4499.000,18035.209,11034.080,1.000,9999.000,17919.000,25060.500,136000.000
2.000,3273.000,41755.299,28291.431,1.000,21550.000,34772.000,54800.000,317000.000
3.000,3674.000,77442.521,39170.143,10.000,48000.000,72914.500,99950.000,291800.000


In [20]:
con1 = (df.km<9999)
con2 = (df.km >= 9999) &(df.km < 25000)
con3 = (df.km >= 25000)&(df.km < 53000)
con4 = (df.km >= 53000)

In [21]:
df.loc[con1, "age"]= df.loc[con1, "age"].fillna(0)
df.loc[con2, "age"]= df.loc[con2, "age"].fillna(1)
df.loc[con3, "age"]= df.loc[con3, "age"].fillna(2)
df.loc[con4, "age"] = df.loc[con4, "age"].fillna(3)

In [22]:
df.age.value_counts(dropna=False)

1.000    4528
3.000    3679
0.000    3597
2.000    3277
NaN       838
Name: age, dtype: int64

In [23]:
df.age.fillna("-",inplace=True)

In [25]:
df.groupby(["make_model","age"]).price.describe()

count      mean      std       min       25%       50%  \
make_model     age                                                            
Audi A1        0.0 577.000 23876.269 3329.721 14900.000 21435.000 22900.000   
               1.0 747.000 18446.178 2607.312 13450.000 16430.000 16950.000   
               2.0 432.000 16625.734 2194.033 10999.000 15000.000 15877.500   
               3.0 629.000 14489.769 1889.865  8999.000 13260.000 13990.000   
               -   229.000 23842.852 3479.884 15980.000 21480.000 22785.000   
Audi A2        1.0   1.000 28200.000      NaN 28200.000 28200.000 28200.000   
Audi A3        0.0 712.000 25045.758 3076.977 15500.000 23700.000 24900.000   
               1.0 777.000 22504.332 5265.565  5555.000 20800.000 21500.000   
               2.0 675.000 19939.557 4004.252 10550.000 18000.000 19290.000   
               3.0 818.000 16466.090 2794.818 10950.000 14999.000 15950.000   
               -   115.000 24172.609 2150.788 18980.000 22921.500 23900.000   
Opel Astra     0.0 559.000 21621.258 5263.130  6950.000 17877.000 18980.000   
               1.0 727.000 16134.623 1948.987   255.000 14900.000 15450.000   
               2.0 587.000 13008.951 2021.046  7800.000 11700.000 12540.000   
               3.0 521.000 11357.733 2503.592  6000.000  9580.000 10750.000   
               -   132.000 19913.439 3855.932 13929.000 16977.000 18594.500   
Opel Corsa     0.0 671.000 13455.006 2695.057   120.000 10979.000 13950.000   
               1.0 684.000 11304.836 2121.837    13.000  9506.000  9950.000   
               2.0 386.000  9249.490 1625.549  6200.000  8357.500  8800.000   
               3.0 430.000  8195.207 1800.657  4950.000  7312.500  7680.000   
               -    48.000 13717.625 3075.090  9245.000 10860.000 13623.500   
Opel Insignia  0.0 450.000 33633.709 4797.618 19900.000 29995.000 33899.000   
               1.0 768.000 21377.004 2120.549 15990.000 19980.000 21285.000   
               2.0 589.000 16995.141 2957.716  8990.000 14980.000 17790.000   
               3.0 568.000 13606.710 2181.531  8290.000 12000.000 13850.000   
               -   223.000 29016.103 5445.378 19902.000 25027.500 28055.000   
Renault Clio   0.0 366.000 11912.385 3114.163  6900.000 10617.500 10990.000   
               1.0 573.000 14698.492 5005.005  7950.000 10390.000 14870.000   
               2.0 438.000 10814.541 2930.586  6000.000  8920.000  9680.000   
               3.0 413.000  9349.533 2666.892  5445.000  7790.000  8450.000   
               -    49.000 11795.000 2223.095  8750.000 10790.000 11100.000   
Renault Duster 0.0  16.000 13343.000 1625.593 10900.000 12162.500 13219.500   
               -    18.000 13129.889 1659.899 10850.000 11925.000 13189.000   
Renault Espace 0.0 246.000 39812.240 8007.526 24200.000 32975.000 38875.000   
               1.0 251.000 31838.781 4033.154 23099.000 29385.000 31900.000   
               2.0 170.000 26371.694 3591.454 14600.000 24802.500 25490.000   
               3.0 300.000 21878.583 3272.034 12614.000 19887.500 21900.000   
               -    24.000 40724.250 6666.727 28650.000 36020.500 40496.500   

                         75%       max  
make_model     age                      
Audi A1        0.0 26890.000 37900.000  
               1.0 20525.000 33900.000  
               2.0 17972.500 23490.000  
               3.0 15750.000 19900.000  
               -   27380.000 29181.000  
Audi A2        1.0 28200.000 28200.000  
Audi A3        0.0 26188.000 63900.000  
               1.0 22900.000 74600.000  
               2.0 20900.000 67600.000  
               3.0 17800.000 42500.000  
               -   25415.000 34738.000  
Opel Astra     0.0 25925.000 39433.000  
               1.0 17960.000 18990.000  
               2.0 14595.000 19950.000  
               3.0 13490.000 17750.000  
               -   22570.000 34900.000  
Opel Corsa     0.0 15889.000 18622.000  
               1.0 13482.500 15790.000  
               2.0  8990.000 13990.000  

In [26]:
df.age.replace("-",0,inplace=True)

In [27]:
df.km.fillna(df.groupby("age").km.transform("mean"),inplace=True)

In [28]:
df.km.isnull().sum()

0

## type

In [29]:
df.type.value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: type, dtype: int64

In [30]:
df[df.type.isnull()].km

2765   115137.000
5237     1652.171
Name: km, dtype: float64

In [31]:
df.groupby("type").km.describe()

,count,mean,std,min,25%,50%,75%,max
type,,,,,,,,
Demonstration,796.000,4965.752,7281.050,3.000,750.000,3000.000,5922.500,78000.000
Employee's car,1011.000,19893.107,14772.496,1.000,9063.000,19998.000,27132.500,112789.000
New,1650.000,868.331,922.414,0.000,10.000,1652.171,1652.171,18026.141
Pre-registered,1364.000,608.853,2667.408,1.000,10.000,10.000,50.000,18035.100
Used,11096.000,43645.969,38332.133,1.000,15900.000,29900.000,63741.500,317000.000


In [32]:
cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)
df.loc[cond1,'type'] = df.loc[cond1,'type'].fillna('New')
df.loc[cond2,'type'] = df.loc[cond2,'type'].fillna('Used')
df.type.value_counts(dropna=False)

Used              11097
New                1651
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: type, dtype: int64

## previous_owners

In [34]:
df.previous_owners.value_counts(dropna=False)

1.000    8294
NaN      6640
2.000     778
0.000     188
3.000      17
4.000       2
Name: previous_owners, dtype: int64

In [35]:
df.groupby("age").previous_owners.value_counts(dropna=False)

age    previous_owners
0.000  NaN                2674
       1.000              1587
       0.000               164
       2.000                10
1.000  1.000              2968
       NaN                1477
       2.000                75
       0.000                 8
2.000  1.000              1711
       NaN                1256
       2.000               296
       3.000                 7
       0.000                 6
       4.000                 1
3.000  1.000              2028
       NaN                1233
       2.000               397
       0.000                10
       3.000                10
       4.000                 1
Name: previous_owners, dtype: int64

In [36]:
df.previous_owners.fillna("-",inplace=True)

In [37]:
df.groupby(["make_model","age","previous_owners"]).price.describe()

count      mean      std       min  \
make_model     age   previous_owners                                        
Audi A1        0.000 0.0              45.000 21968.556 3086.935 17173.000   
                     1.0             239.000 24217.163 3479.484 14900.000   
                     2.0               1.000 21760.000      NaN 21760.000   
                     -               521.000 23874.036 3296.457 15980.000   
               1.000 0.0               1.000 19700.000      NaN 19700.000   
                     1.0             496.000 18130.627 2405.771 13500.000   
                     2.0              14.000 19293.929 2629.516 14960.000   
                     -               236.000 19053.767 2894.165 13450.000   
               2.000 0.0               1.000 15900.000      NaN 15900.000   
                     1.0             195.000 16774.631 2329.836 10999.000   
                     2.0              74.000 16429.189 1612.890 13390.000   
                     3.0               1.000 15480.000      NaN 15480.000   
                     4.0               1.000 21300.000      NaN 21300.000   
                     -               160.000 16517.650 2241.742 11990.000   
               3.000 0.0               4.000 13522.500 1736.095 11790.000   
                     1.0             401.000 14425.212 1874.603  9950.000   
                     2.0              82.000 14604.024 1450.092 10800.000   
                     3.0               1.000 11990.000      NaN 11990.000   
                     -               141.000 14652.092 2140.576  8999.000   
Audi A2        1.000 1.0               1.000 28200.000      NaN 28200.000   
Audi A3        0.000 0.0              49.000 24700.245 2863.419 20000.000   
                     1.0             257.000 25232.432 2973.242 20990.000   
                     -               521.000 24793.441 2987.797 15500.000   
               1.000 0.0               5.000 20340.000  502.991 19800.000   
                     1.0             380.000 22320.576 4735.775 14250.000   
                     2.0              11.000 21669.091  928.498 20200.000   
                     -               381.000 22740.123 5835.627  5555.000   
               2.000 1.0             351.000 19632.746 3827.776 13000.000   
                     2.0              47.000 20318.787 3335.756 16990.000   
                     -               277.000 20263.986 4297.783 10550.000   
               3.000 0.0               1.000 15899.000      NaN 15899.000   
                     1.0             458.000 16191.245 2081.439 10950.000   
                     2.0              53.000 16049.038 1424.293 11990.000   
                     3.0               2.000 15500.000 3394.113 13100.000   
                     -               304.000 16961.099 3707.182 12400.000   
Opel Astra     0.000 0.0              12.000 20265.417 5736.997 14150.000   
                     1.0             253.000 21579.312 4878.367  6950.000   
                     2.0               1.000 17990.000      NaN 17990.000   
                     -               425.000 21162.626 5163.663 13929.000   
               1.000 0.0               1.000 13900.000      NaN 13900.000   
                     1.0             531.000 16426.942 1837.095  8860.000   
                     2.0              24.000 15629.000 1974.526 12978.000   
                     -               171.000 15310.930 2039.100   255.000   
               2.000 1.0             354.000 12692.876 1946.030  7800.000   
                     2.0              54.000 14881.426 1673.193 10490.000   
                     3.0               1.000 17400.000      NaN 17400.000   
                     -               178.000 13044.826 1937.123  8800.000   
               3.000 1.0             349.000 11172.501 2417.853  6000.000   
                     2.0              55.000 12901.255 2733.922  6480.000   
                     3.0               2.000 10744.000  206.475 10598.000   
                     -             

In [38]:
df.make_model.value_counts(dropna=False)

Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: make_model, dtype: int64

In [39]:
df.loc[(df["make_model"]== "Audi A1")&(df.age ==3)]["previous_owners"] = df.loc[(df["make_model"]== "Audi A1")&(df.age ==3)].previous_owners.replace("-",2)
df.loc[(df.make_model== "Audi A3")&(df.age==0)]["previous_owners"] = df.loc[(df.make_model== "Audi A3")&(df.age==0)].previous_owners.replace("-",0)
df.loc[(df.make_model== "Audi A3")&(df.age==2)]["previous_owners"] = df.loc[(df.make_model== "Audi A3")&(df.age==2)].previous_owners.replace("-",2)
df.loc[(df.make_model== "Opel Astra")&(df.age==1)]["previous_owners"] = df.loc[(df.make_model== "Opel Astra")&(df.age==1)].previous_owners.replace("-",2)
df.loc[(df.make_model== "Opel Corsa")&(df.age==0)]["previous_owners"] = df.loc[(df.make_model== "Opel Corsa")&(df.age==0)].previous_owners.replace("-",0)
df.loc[(df.make_model== "Opel Corsa")&(df.age==2)]["previous_owners"] = df.loc[(df.make_model== "Opel Corsa")&(df.age==2)].previous_owners.replace("-",3)
df.loc[(df.make_model== "Opel Insignia")&(df.age==2)]["previous_owners"] = df.loc[(df.make_model== "Opel Insignia")&(df.age==2)].previous_owners.replace("-",0)
df.loc[(df.make_model== "Opel Insignia")&(df.age==1)]["previous_owners"] = df.loc[(df.make_model== "Opel Insignia")&(df.age==1)].previous_owners.replace("-",2)
df.loc[df.make_model== "Renault Duster"]["previous_owners"] = df.loc[df.make_model== "Renault Duster"].previous_owners.replace("-",0)
df.loc[(df.make_model== "Renault Espace")&(df.age==0)]["previous_owners"] = df.loc[(df.make_model== "Renault Espace")&(df.age==0)].previous_owners.replace("-",0)
df.loc[(df.make_model== "Renault Espace")&(df.age==2)]["previous_owners"] = df.loc[(df.make_model== "Renault Espace")&(df.age==2)].previous_owners.replace("-",2)

In [40]:
df.previous_owners.replace("-",1,inplace=True)
df.previous_owners.value_counts(dropna=False)

1.000    14934
2.000      778
0.000      188
3.000       17
4.000        2
Name: previous_owners, dtype: int64

## inspection_new

In [41]:
df.inspection_new.value_counts(dropna=False)

NaN    11987
Yes     3932
Name: inspection_new, dtype: int64

In [42]:
df.inspection_new.fillna("No", inplace=True)

In [43]:
df.inspection_new.replace(["Yes","No"],[1,0],inplace=True)

In [44]:
df.inspection_new.value_counts(dropna=False)

0    11987
1     3932
Name: inspection_new, dtype: int64

## body_color

In [45]:
df.body_color.value_counts(dropna=False)

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: body_color, dtype: int64

In [46]:
df[df.age == 0].groupby(["make_model","body_color"]).price.describe()

count      mean      std       min       25%  \
make_model     body_color                                                  
Audi A1        Black      161.000 23680.000 3359.070 15550.000 21390.000   
               Blue        82.000 24578.415 2881.486 17900.000 22505.000   
               Green       29.000 23093.724 3224.056 19388.000 20900.000   
               Grey       103.000 25166.330 2982.836 19450.000 22515.000   
               Red         65.000 24667.108 3121.172 18480.000 21900.000   
               Silver       4.000 23012.500 3086.458 20420.000 20840.000   
               White      287.000 23312.460 3324.301 14900.000 20990.000   
               Yellow      28.000 26780.964 2998.861 19900.000 25625.000   
Audi A3        Black      246.000 24826.285 3287.331 15500.000 23500.000   
               Blue        40.000 25470.050 1791.761 20000.000 24262.500   
               Grey       160.000 25648.206 4126.526 17990.000 23988.000   
               Red         22.000 24273.500 1874.750 20300.000 22413.500   
               Silver      80.000 25116.013 2427.813 21900.000 23500.000   
               Violet       1.000 25740.000      NaN 25740.000 25740.000   
               White      224.000 24736.772 1984.638 19078.000 23600.000   
Opel Astra     Black      151.000 22564.616 5566.867 15455.000 17990.000   
               Blue        78.000 21671.282 5263.547 13945.000 17800.000   
               Brown       15.000 18827.133 3346.145 15990.000 17195.000   
               Green        9.000 24915.000 4392.353 17600.000 23595.000   
               Grey       125.000 22498.976 5280.994  6950.000 17990.000   
               Red         44.000 19602.886 5101.789 14430.000 16957.750   
               Silver      63.000 21593.746 5171.396 14555.000 17915.000   
               White       94.000 19735.723 4673.883 14150.000 16977.000   
Opel Corsa     Black      109.000 14052.890 2454.505 10470.000 11169.000   
               Blue       170.000 13002.076 2712.633  9245.000 10689.000   
               Green        1.000 16990.000      NaN 16990.000 16990.000   
               Grey       123.000 14771.171 2552.764   120.000 13663.000   
               Red         67.000 12479.672 2545.373  9580.000 10770.000   
               Silver      83.000 14753.723 2330.660  8200.000 13288.000   
               White      145.000 12462.303 2623.469   331.000 10940.000   
               Yellow       1.000 10580.000      NaN 10580.000 10580.000   
Opel Insignia  Beige        2.000 32160.500 6520.232 27550.000 29855.250   
               Black      137.000 34821.394 4305.665 24950.000 31990.000   
               Blue        68.000 32405.191 5688.983 20580.000 27561.750   
               Brown       17.000 31802.529 4589.169 26890.000 29995.000   
               Green        2.000 32220.000 8810.550 25990.000 29105.000   
               Grey       120.000 34184.142 4706.437 19900.000 31444.250   
               Orange       1.000 37950.000      NaN 37950.000 37950.000   
               Red         11.000 34048.545 3146.975 29790.000 31895.000   
               Silver      30.000 32085.467 4659.122 22990.000 29568.250   
               White      100.000 34135.850 4503.942 21900.000 31987.500   
Renault Clio   Black       61.000 12483.557 3998.598  6900.000 10390.000   
               Blue        32.000 11285.719 3566.650  6950.000 10197.500   
               Brown        1.000 10390.000      NaN 10390.000 10390.000   
               Grey        69.000 12860.145 3113.163  9999.000 10990.000   
               Red         40.000 12164.750 3264.633  6950.000 10990.000   
               Silver      20.000 11365.450 1848.510  9850.000 10990.000   
               White      158.000 11035.013 1581.555  9262.000 10241.250   
               Yellow       3.000 24347.667 1549.831 22600.000 23744.000   
Renault Duster Black        8.000 14641.500 1487.576 13250.000 13250.000   
               White       26.000 12795.923 1421.755 10850.000 11900.000   


It seems not to be significant difference between color and price. So let's drop the column.

In [47]:
df.drop("body_color",axis=1,inplace=True)

## paint type

In [48]:
df.paint_type.value_counts(dropna=False)

Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: paint_type, dtype: int64

In [49]:
df.paint_type.fillna("-",inplace=True)

In [50]:
df[df.age==0].groupby(["make_model","paint_type"]).price.describe()

count      mean      std       min       25%  \
make_model     paint_type                                                   
Audi A1        -           287.000 22619.847 3490.381 14900.000 20189.500   
               Metallic    500.000 24551.016 3089.782 15990.000 21999.000   
               Perl effect   1.000 28290.000      NaN 28290.000 28290.000   
               Uni/basic    18.000 24495.889 3362.339 19888.000 21490.000   
Audi A3        -           378.000 24511.749 2781.762 17990.000 23212.500   
               Metallic    415.000 25297.106 3208.269 15500.000 23900.000   
               Uni/basic    34.000 24961.471 1011.605 22980.000 24067.500   
Opel Astra     -           260.000 20049.288 4370.008  6950.000 17232.500   
               Metallic    421.000 22102.276 5343.233 14555.000 17900.000   
               Perl effect   2.000 17745.000  346.482 17500.000 17622.500   
               Uni/basic     8.000 20186.750 3069.024 16464.000 18174.750   
Opel Corsa     -           302.000 11974.162 2345.513   331.000 10689.000   
               Metallic    401.000 14667.052 2401.084   120.000 13663.000   
               Uni/basic    16.000 11816.875 1964.481  8800.000 10923.000   
Opel Insignia  -           281.000 29454.548 5200.256 19902.000 25375.000   
               Metallic    377.000 34131.509 4823.822 19900.000 30800.000   
               Uni/basic    15.000 30763.533 4021.281 24999.000 28009.000   
Renault Clio   -           226.000 11510.367 2461.765  6950.000 10467.500   
               Metallic    172.000 12455.436 3655.565  6900.000 10785.000   
               Uni/basic    17.000 11424.118 1535.135  9850.000 10990.000   
Renault Duster -            17.000 13196.353 1686.115 10850.000 11900.000   
               Metallic      1.000 12000.000      NaN 12000.000 12000.000   
               Uni/basic    16.000 13343.000 1625.593 10900.000 12162.500   
Renault Espace -           120.000 35686.367 5711.305 26450.000 31900.000   
               Metallic    149.000 43386.772 7658.135 28890.000 37990.000   
               Uni/basic     1.000 24200.000      NaN 24200.000 24200.000   

                                 50%       75%       max  
make_model     paint_type                                 
Audi A1        -           21900.000 25100.000 37900.000  
               Metallic    24490.000 27380.000 32000.000  
               Perl effect 28290.000 28290.000 28290.000  
               Uni/basic   24985.000 27675.000 29000.000  
Audi A3        -           24500.000 25500.000 62900.000  
               Metallic    24900.000 26488.000 63900.000  
               Uni/basic   24875.000 25772.500 26999.000  
Opel Astra     -           18410.000 22695.000 39433.000  
               Metallic    18990.000 26441.000 37995.000  
               Perl effect 17745.000 17867.500 17990.000  
               Uni/basic   18830.500 22715.000 24790.000  
Opel Corsa     -           10990.000 12930.000 18490.000  
               Metallic    15390.000 16390.000 18622.000  
               Uni/basic   11294.000 11975.000 15390.000  
Opel Insignia  -           28990.000 33895.000 42475.000  
               Metallic    33990.000 37890.000 52120.000  
               Uni/basic   29702.000 33650.000 37599.000  
Renault Clio   -           10990.000 11380.000 26000.000  
               Metallic    11199.000 11450.000 26970.000  
               Uni/basic   10990.000 11480.000 17200.000  
Renault Duster -           13189.000 14399.000 16033.000  
               Metallic    12000.000 12000.000 12000.000  
               Uni/basic   13219.500 14505.000 16033.000  
Renault Espace -           33400.000 38881.250 54205.000  
               Metallic    42990.000 47990.000 68320.000  
               Uni/basic   24200.000 24200.000 24200.000

It seems that Metallic paint type is more expensive than uni/basic. pearl effect is very rare.
So we can replace expensive ones with "metallic" and cheaper ones with "uni/basic"

In [51]:
for model, age in zip(df.make_model.unique(), df.age.unique()):
    if df[(df.make_model == model)&(df.age == age)].price.mean() > df[(df.make_model == model)&(df.age == age)&(df.paint_type == "-")].price.mean():
        df.loc[(df.make_model == model)&(df.age == age),"paint_type"] = df.loc[(df.make_model == model)&(df.age == age),"paint_type"].replace("-","Uni/basic")

In [52]:
df.paint_type.replace("-","Metallic",inplace=True)

In [53]:
df.paint_type.value_counts(dropna=False)

Metallic       15093
Uni/basic        820
Perl effect        6
Name: paint_type, dtype: int64

## nr_of_doors & nr_of_seats
we can fill them with each other.

In [54]:
df.nr_of_doors.value_counts(dropna=False)  

5.000    11575
4.000     3079
3.000      832
2.000      219
NaN        212
7.000        1
1.000        1
Name: nr_of_doors, dtype: int64

In [55]:
df[df.nr_of_doors.isnull()].nr_of_seats.value_counts(dropna=False)

NaN      168
5.000     35
7.000      6
4.000      2
2.000      1
Name: nr_of_seats, dtype: int64

In [56]:
df.groupby("nr_of_seats").nr_of_doors.value_counts()

nr_of_seats  nr_of_doors
2.000        5.000           107
             3.000             5
             4.000             3
3.000        2.000             1
4.000        5.000           697
             3.000           196
             4.000           130
             2.000           100
5.000        5.000          9847
             4.000          2737
             3.000           605
             2.000           112
6.000        5.000             2
7.000        5.000           295
             4.000            61
Name: nr_of_doors, dtype: int64

there seems to be no relationship. so we can use make_model and body_type

In [57]:
df.nr_of_doors.fillna(df.groupby(["make_model", "body_type"]).nr_of_doors.transform("median"),inplace=True)

In [58]:
df.nr_of_seats.fillna(df.groupby(["make_model", "body_type"]).nr_of_seats.transform("median"),inplace=True)

In [59]:
df.nr_of_doors.value_counts(dropna=False)

5.000    11776
4.000     3088
3.000      832
2.000      219
NaN          2
7.000        1
1.000        1
Name: nr_of_doors, dtype: int64

In [60]:
df.nr_of_seats.value_counts(dropna=False)

5.000    14302
4.000     1127
7.000      362
2.000      119
NaN          6
6.000        2
3.000        1
Name: nr_of_seats, dtype: int64

In [61]:
df[df.nr_of_seats.isnull()][["nr_of_doors","make_model"]]

,nr_of_doors,make_model
2382,5.000,Audi A1
8952,NaN,Opel Corsa
8953,NaN,Opel Corsa
11620,4.000,Opel Insignia
15008,5.000,Renault Espace
15785,5.000,Renault Espace


In [62]:
df.nr_of_seats.fillna(5, inplace=True)
df.nr_of_doors.fillna(5, inplace=True)

## cylinders

NameError: name 'cons_comb' is not defined

In [64]:
df.cylinders.value_counts(dropna=False)

4.000    8105
NaN      5680
3.000    2104
5.000      22
6.000       3
8.000       2
2.000       2
1.000       1
Name: cylinders, dtype: int64

In [65]:
df.groupby("cylinders").cons_comb.describe()

,count,mean,std,min,25%,50%,75%,max
cylinders,,,,,,,,
1.000,1.000,6.000,NaN,6.000,6.000,6.000,6.000,6.000
2.000,1.000,5.300,NaN,5.300,5.300,5.300,5.300,5.300
3.000,1993.000,4.636,2.293,3.000,4.200,4.600,4.800,51.000
4.000,7524.000,4.834,1.307,0.000,3.900,4.700,5.500,55.000
5.000,21.000,8.152,0.588,5.600,8.300,8.300,8.300,8.300
6.000,2.000,7.250,2.616,5.400,6.325,7.250,8.175,9.100
8.000,1.000,13.800,NaN,13.800,13.800,13.800,13.800,13.800


In [66]:
df.groupby("cylinders").hp_kW.describe()

,count,mean,std,min,25%,50%,75%,max
cylinders,,,,,,,,
1.000,1.000,66.000,NaN,66.000,66.000,66.000,66.000,66.000
2.000,2.000,123.500,33.234,100.000,111.750,123.500,135.250,147.000
3.000,2103.000,73.835,9.760,51.000,66.000,70.000,85.000,110.000
4.000,8095.000,91.600,26.756,44.000,70.000,85.000,110.000,228.000
5.000,22.000,271.409,65.902,54.000,294.000,294.000,294.000,294.000
6.000,3.000,100.000,0.000,100.000,100.000,100.000,100.000,100.000
8.000,2.000,93.500,33.234,70.000,81.750,93.500,105.250,117.000


In [67]:
df.groupby("cylinders").displacement_cc.describe()

,count,mean,std,min,25%,50%,75%,max
cylinders,,,,,,,,
1.000,1.000,1398.000,NaN,1398.000,1398.000,1398.000,1398.000,1398.000
2.000,1.000,1997.000,NaN,1997.000,1997.000,1997.000,1997.000,1997.000
3.000,2102.000,1052.527,156.343,890.000,999.000,999.000,999.000,1598.000
4.000,8063.000,1534.820,261.890,1.000,1398.000,1598.000,1598.000,15898.000
5.000,21.000,2438.000,192.468,1598.000,2480.000,2480.000,2480.000,2480.000
6.000,3.000,2055.000,789.816,1598.000,1599.000,1600.000,2283.500,2967.000
8.000,1.000,999.000,NaN,999.000,999.000,999.000,999.000,999.000


In [68]:
df.groupby("cylinders").weight_kg.describe()

,count,mean,std,min,25%,50%,75%,max
cylinders,,,,,,,,
1.000,1.000,1163.000,NaN,1163.000,1163.000,1163.000,1163.000,1163.000
2.000,1.000,1840.000,NaN,1840.000,1840.000,1840.000,1840.000,1840.000
3.000,1421.000,1236.347,170.470,102.000,1135.000,1195.000,1260.000,1790.000
4.000,6243.000,1381.875,218.020,1.000,1209.000,1350.000,1503.000,2471.000
5.000,21.000,1555.667,101.145,1134.000,1585.000,1585.000,1585.000,1595.000
6.000,1.000,2295.000,NaN,2295.000,2295.000,2295.000,2295.000,2295.000
8.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cylinders generally give us clues about weight, hp, displacement, consumption etc. we have all of them. so we can drop this column.

In [69]:
df.drop("cylinders", axis=1, inplace=True)

## drive chain

In [70]:
df[df.age==1].groupby(["make_model","body_type","drive_chain"]).price.describe()

count      mean      std       min  \
make_model     body_type     drive_chain                                        
Audi A1        Compact       front       200.000 18773.970 2630.799 13980.000   
               Sedans        front       317.000 18346.290 2410.602 14490.000   
               Station wagon front        10.000 17563.000 2220.421 15750.000   
Audi A2        Off-Road      front         1.000 28200.000      NaN 28200.000   
Audi A3        Compact       front        26.000 21832.615 4173.638 14500.000   
               Sedans        4WD           5.000 59958.000 8604.523 51900.000   
                             front       497.000 21702.622 1538.864 17996.000   
               Station wagon front        17.000 21728.706 1158.946 19900.000   
               Convertible   4WD           1.000 56100.000      NaN 56100.000   
                             front         1.000 10893.000      NaN 10893.000   
Opel Astra     Compact       front        11.000 14426.727 2315.534 11999.000   
               Sedans        4WD           1.000 15650.000      NaN 15650.000   
                             front       135.000 16029.119 1860.889  8860.000   
               Station wagon front       174.000 17009.052 1729.661 12450.000   
               Off-Road      front         1.000 18590.000      NaN 18590.000   
Opel Corsa     Compact       front       147.000 12729.599 1838.333  7499.000   
               Sedans        front       161.000 11064.155 2006.148  7990.000   
Opel Insignia  Sedans        front       147.000 20174.327 2190.535 15990.000   
               Station wagon 4WD           1.000 22550.000      NaN 22550.000   
                             front       245.000 22183.522 1605.189 17500.000   
                             rear          1.000 18950.000      NaN 18950.000   
               Van           front         1.000 20980.000      NaN 20980.000   
Renault Clio   Compact       front        62.000 16397.839 4009.897  9590.000   
               Sedans        4WD           9.000 23021.000    0.000 23021.000   
                             front       115.000 14188.774 4745.665  8700.000   
                             rear          1.000 10490.000      NaN 10490.000   
               Station wagon front        51.000 10602.745 1433.431  7950.000   
               Transporter   front         1.000  9885.000      NaN  9885.000   
Renault Espace Sedans        front         7.000 32063.143 1700.772 29490.000   
               Station wagon 4WD           1.000 35990.000      NaN 35990.000   
                             front         9.000 34942.222 4360.452 28990.000   
               Van           4WD           8.000 32391.250 3511.595 24990.000   
                             front        97.000 31629.464 4196.786 23099.000   
               Transporter   4WD           1.000 30000.000      NaN 30000.000   
                             front         1.000 27999.000      NaN 27999.000   

                                               25%       50%       75%  \
make_model     body_type     drive_chain                                 
Audi A1        Compact       front       16445.000 17630.000 20958.750   
               Sedans        front       16440.000 16950.000 19990.000   
               Station wagon front       15960.000 16490.000 18950.000   
Audi A2        Off-Road      front       28200.000 28200.000 28200.000   
Audi A3        Compact       front       19999.250 21700.000 22300.000   
               Sedans        4WD         56900.000 57400.000 58990.000   
                             front       20800.000 21500.000 22600.000   
               Station wagon front       20700.000 21480.000 22550.000   
               Convertible   4WD         56100.000 56100.000 56100.000   
                             front       10893.000 10893.000 10893.000   
Opel Astra     Compact       front       13444.500 13899.000 13949.000   
               Sedans        4WD         15650.000 15650.000 15650.000   
         

In [71]:
df.groupby(["body_type","drive_chain"]).price.describe()

count      mean       std       min       25%  \
body_type     drive_chain                                                    
Compact       4WD            4.000 26715.000 27372.553  9680.000 12845.000   
              front       1800.000 16106.466  5683.653  4990.000 12192.500   
Convertible   4WD            1.000 56100.000       NaN 56100.000 56100.000   
              front          3.000 20897.667 10003.500 10893.000 15896.500   
Coupe         front         11.000 11893.636  4968.181  6900.000  8150.000   
Off-Road      4WD            7.000 23430.571 10988.959 10850.000 12375.000   
              front          7.000 22769.286  6009.723 14900.000 18290.000   
Sedans        4WD           55.000 30835.382 13254.097 10500.000 22760.000   
              front       4711.000 18253.127  5935.569  5445.000 13990.000   
              rear           2.000 14740.000  6010.408 10490.000 12615.000   
Station wagon 4WD           65.000 27726.369 10664.481 11480.000 16900.000   
              front       1959.000 19409.043  7525.614  5900.000 13900.000   
              rear           2.000 15925.000  4277.996 12900.000 14412.500   
Transporter   4WD            1.000 30000.000       NaN 30000.000 30000.000   
              front         20.000 12548.400  7954.381  5250.000  7337.500   
Van           4WD           38.000 38150.763 10013.779 24950.000 31900.000   
              front        375.000 31434.917  8425.505  9400.000 24999.000   

                                50%       75%       max  
body_type     drive_chain                                
Compact       4WD         14790.000 28660.000 67600.000  
              front       15485.000 19422.500 41495.000  
Convertible   4WD         56100.000 56100.000 56100.000  
              front       20900.000 25900.000 30900.000  
Coupe         front        8800.000 17490.000 19000.000  
Off-Road      4WD         28500.000 32569.000 34776.000  
              front       22500.000 26950.000 31505.000  
Sedans        4WD         29070.000 37028.000 74600.000  
              front       18290.000 22422.500 49350.000  
              rear        14740.000 16865.000 18990.000  
Station wagon 4WD         34550.000 36950.000 42475.000  
              front       18070.000 23467.500 52990.000  
              rear        15925.000 17437.500 18950.000  
Transporter   4WD         30000.000 30000.000 30000.000  
              front        9431.000 17462.500 32700.000  
Van           4WD         34200.000 45747.000 57990.000  
              front       31265.000 36151.000 68320.000

In [72]:
df.drive_chain.value_counts(dropna=False)

front    8886
NaN      6858
4WD       171
rear        4
Name: drive_chain, dtype: int64

In [73]:
df.loc[df.body_type=="Coupe", "drive_chain"] = df.loc[df.body_type=="Coupe", "drive_chain"].fillna("front")
df.drive_chain.value_counts(dropna=False)

front    8900
NaN      6844
4WD       171
rear        4
Name: drive_chain, dtype: int64

In [74]:
df.groupby(["make_model","body_type"]).drive_chain.value_counts(dropna=False)

make_model      body_type      drive_chain
Audi A1         Compact        front           685
                               NaN             352
                               4WD               2
                Coupe          front             2
                Sedans         front           989
                               NaN             561
                               4WD               1
                Station wagon  front            18
                               NaN               3
                Van            front             1
Audi A2         Off-Road       front             1
Audi A3         Compact        front           156
                               NaN              25
                               4WD               1
                Convertible    NaN               4
                               front             3
                               4WD               1
                Coupe          front             4
                Sedans         front   

In [75]:
df.loc[(df.make_model=="Audi A1")&(df.body_type=="Station wagon")].drive_chain = df.loc[(df.make_model=="Audi A1")&(df.body_type=="Station wagon")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Audi A3")&(df.body_type=="Station wagon")].drive_chain = df.loc[(df.make_model=="Audi A3")&(df.body_type=="Station wagon")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make=="Opel")&(df.body_type=="Compact")].drive_chain = df.loc[(df.make =="Opel")&(df.body_type=="Compact")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Opel Astra")&(df.body_type=="Station wagon")].drive_chain = df.loc[(df.make_model=="Opel Astra")&(df.body_type=="Station wagon")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Opel Corsa")].drive_chain = df.loc[(df.make_model=="Opel Corsa")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Renault Clio")&(df.body_type == "Off-Road")].drive_chain = df.loc[(df.make_model=="Renault Clio")&(df.body_type == "Off-Road")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Renault Clio")&(df.body_type == "Transporter")].drive_chain = df.loc[(df.make_model=="Renault Clio")&(df.body_type =="Transporter")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Renault Clio")&(df.body_type ==  "Van")].drive_chain = df.loc[(df.make_model=="Renault Clio")&(df.body_type =="Van")].drive_chain.fillna("front",inplace=True)
df.loc[df.make_model=="Renault Duster"].drive_chain = df.loc[df.make_model=="Renault Duster"].drive_chain.fillna("4WD",inplace=True)
df.loc[(df.make_model=="Renault Espace")&(df.body_type == "Compact" )].drive_chain = df.loc[(df.make_model=="Renault Espace")&(df.body_type =="Compact")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Renault Espace")&(df.body_type ==  "Off-Road")].drive_chain = df.loc[(df.make_model=="Renault Espace")&(df.body_type =="Off-Road")].drive_chain.fillna("front",inplace=True)
df.loc[(df.make_model=="Renault Espace")&(df.body_type =="Sedans")].drive_chain = df.loc[(df.make_model=="Renault Espace")&(df.body_type =="Sedans")].drive_chain.fillna("front",inplace=True)

In [76]:
df.drive_chain.value_counts(dropna=False)

front    8900
NaN      6844
4WD       171
rear        4
Name: drive_chain, dtype: int64

In [77]:
def fill(df, group_col1, group_col2, col_name, method): # method can be either "mode" or "mean" or "median" or "ffill"
    
    '''Fills the missing values with "mode/mean/median/ffill/bfill method" according to double-stage grouping'''
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

This code is very useful. I keep it for other EDA projects.

In [78]:
fill(df, "make_model", "body_type", "drive_chain", "mode")

Number of NaN :  0
------------------
front    15711
4WD        204
rear         4
Name: drive_chain, dtype: int64


## co2_emission
CO2 emission can be related to make_model and fuel type.

In [79]:
df.co2_emission.value_counts(dropna=False)

NaN          2436
120.000       740
99.000        545
97.000        537
104.000       501
102.000       477
103.000       445
114.000       382
124.000       372
107.000       362
108.000       362
119.000       361
106.000       349
128.000       329
126.000       282
85.000        275
118.000       270
110.000       266
127.000       257
117.000       254
111.000       237
113.000       235
109.000       234
139.000       224
140.000       218
129.000       213
135.000       202
105.000       202
92.000        198
130.000       180
123.000       178
150.000       174
112.000       171
143.000       171
95.000        161
116.000       157
141.000       156
98.000        151
133.000       145
136.000       145
137.000       133
125.000       132
134.000       130
145.000       126
149.000       117
153.000       113
147.000       109
101.000       105
132.000       100
115.000        86
122.000        83
121.000        82
138.000        75
93.000         66
142.000        58
168.000   

In [84]:
df.groupby(["make_model","fuel"])["co2_emission"].describe()

count    mean     std     min     25%     50%  \
make_model     fuel                                                        
Audi A1        benzine  1590.000 106.337   9.369  97.000 102.000 105.000   
               diesel    714.000 100.101   4.687  89.000  97.000  99.000   
Audi A2        diesel      0.000     NaN     NaN     NaN     NaN     NaN   
Audi A3        Electric    3.000  36.000   0.000  36.000  36.000  36.000   
               LPG        30.000  97.000   8.785  91.000  91.000  94.000   
               benzine   812.000 116.421  13.212  96.000 110.000 117.000   
               diesel   1903.000 106.831  30.471  85.000 102.000 105.000   
Opel Astra     LPG         1.000 113.000     NaN 113.000 113.000 113.000   
               benzine  1152.000 154.456 610.175   0.000 114.000 125.000   
               diesel    934.000 105.741  12.881   5.000  95.000 101.000   
Opel Corsa     LPG        12.000 117.667   7.970 110.000 113.000 113.000   
               benzine  1848.000 136.407 283.886  51.000 123.000 128.000   
               diesel     90.000  99.456   8.675  82.000 100.000 101.000   
Opel Insignia  benzine   773.000 176.918 715.917  45.000 133.000 139.000   
               diesel   1326.000 132.254  20.481   0.000 114.500 135.000   
Renault Clio   Electric    0.000     NaN     NaN     NaN     NaN     NaN   
               LPG         8.000  98.000   0.000  98.000  98.000  98.000   
               benzine  1018.000 122.653   8.347  94.000 114.000 120.000   
               diesel    485.000  89.530   7.410  82.000  85.000  85.000   
Renault Duster benzine    33.000 171.667  34.375   1.000 150.000 186.000   
Renault Espace benzine   236.000 155.903  10.573 123.000 153.000 153.000   
               diesel    515.000 124.425  10.403 116.000 120.000 120.000   

                            75%       max  
make_model     fuel                        
Audi A1        benzine  111.000   331.000  
               diesel   104.000   110.000  
Audi A2        diesel       NaN       NaN  
Audi A3        Electric  36.000    36.000  
               LPG       97.000   122.000  
               benzine  118.000   194.000  
               diesel   108.000  1060.000  
Opel Astra     LPG      113.000   113.000  
               benzine  130.000 12087.000  
               diesel   119.000   144.000  
Opel Corsa     LPG      124.000   135.000  
               benzine  139.000 12324.000  
               diesel   104.000   137.000  
Opel Insignia  benzine  145.000 14457.000  
               diesel   146.000   239.000  
Renault Clio   Electric     NaN       NaN  
               LPG       98.000    98.000  
               benzine  127.000   140.000  
               diesel    92.000   136.000  
Renault Duster benzine  186.000   186.000  
Renault Espace benzine  168.000   194.000  
               diesel   123.000   191.000

In [83]:
df.fuel.replace("others","benzine",inplace=True)

In [85]:
fill(df, "make_model", "fuel", "co2_emission", "median")

Number of NaN :  0
------------------
120.000      1081
105.000       656
99.000        617
135.000       591
97.000        537
128.000       535
104.000       501
102.000       477
85.000        459
103.000       445
114.000       383
117.000       381
101.000       372
124.000       372
125.000       365
108.000       362
107.000       362
119.000       361
106.000       349
139.000       334
126.000       282
118.000       270
110.000       266
127.000       257
113.000       237
111.000       237
109.000       234
140.000       218
129.000       213
92.000        198
130.000       180
123.000       178
150.000       174
143.000       171
112.000       171
95.000        161
116.000       157
141.000       156
98.000        156
153.000       150
133.000       145
136.000       145
137.000       133
134.000       130
145.000       126
149.000       117
147.000       109
132.000       100
115.000        86
122.000        83
121.000        82
138.000        75
93.000         66
168.000 

we can see some outliers. we didn't clean them yet. Not to be affected by the outliers i prefer median to mean. 

## emission_class

In [89]:
df.emission_class.fillna("-",inplace=True)

In [94]:
df.groupby(["make_model","body_type","age", "emission_class"])["co2_emission"].describe()

count    mean      std  \
make_model     body_type     age   emission_class                            
Audi A1        Compact       0.000 -               73.000 107.342    4.679   
                                   Euro 6         303.000 109.927    5.591   
                             1.000 -               44.000 102.773    4.778   
                                   Euro 5           4.000  99.250    2.217   
                                   Euro 6         220.000 104.009    5.691   
                             2.000 -               28.000 104.071    5.213   
                                   Euro 5           1.000 102.000      NaN   
                                   Euro 6         132.000 106.030    9.000   
                             3.000 -               37.000 101.568    6.030   
                                   Euro 4           1.000 331.000      NaN   
                                   Euro 6         196.000 103.240    6.634   
               Coupe         2.000 -                1.000  99.000      NaN   
                             3.000 Euro 6           1.000  97.000      NaN   
               Sedans        0.000 -              192.000 107.172    3.507   
                                   Euro 6         235.000 108.749    6.189   
                             1.000 -               70.000 103.500    5.132   
                                   Euro 6         398.000 101.927    5.872   
                             2.000 -               57.000 103.807    6.812   
                                   Euro 6         210.000 101.014    6.995   
                             3.000 -               55.000 102.091    4.628   
                                   Euro 5           4.000  99.750    7.805   
                                   Euro 6         330.000  99.455    6.177   
               Station wagon 0.000 -                1.000 106.000      NaN   
                                   Euro 6           1.000 111.000      NaN   
                             1.000 Euro 6          11.000 103.091    4.182   
                             2.000 Euro 6           3.000 106.000    7.937   
                             3.000 -                1.000  91.000      NaN   
                                   Euro 6           4.000 103.750    9.743   
               Van           0.000 -                1.000 105.000      NaN   
Audi A2        Off-Road      1.000 Euro 6           1.000 117.000      NaN   
Audi A3        Compact       0.000 -               14.000 113.357    5.542   
                                   Euro 6           7.000 111.286    4.536   
                             1.000 -                6.000 107.167    4.834   
                                   Euro 6          23.000 107.304    3.111   
                             2.000 -               10.000 105.000    0.000   
                                   Euro 6          29.000 106.966   16.428   
                             3.000 -               21.000 108.810    7.427   
                                   Euro 6          72.000 109.986   12.817   
               Convertible   1.000 Euro 6           3.000 134.333   18.502   
                             2.000 -                1.000 117.000      NaN   
                                   Euro 6           3.000 113.333   12.897   
                             3.000 Euro 6           1.000 124.000      NaN   
               Coupe         2.000 Euro 6           1.000 106.000      NaN   
                             3.000 Euro 6           3.000  99.000    0.000   
               Sedans        0.000 -              193.000 110.948    6.241   
                                   Euro 6         502.000 114.291    8.423   
                             1.000 -               53.000 145.358  176.096   
                                   Euro 5           1.000 108.000      NaN   
                                   Euro 6         659.000 107.854   12.734   
                             2.000 -               64.000 106.812    4.966

In [91]:
df.loc[df.make_model== "Renault Duster", "emission_class"] = df.loc[df.make_model== "Renault Duster", "emission_class"].replace("-","Euro 4")

In [95]:
df.emission_class.replace("-", np.nan, inplace=True)
fill(df, "make_model", "body_type", "emission_class", "mode")

Number of NaN :  0
------------------
Euro 6    15799
Euro 5       78
Euro 4       42
Name: emission_class, dtype: int64


## comfort_convenience 

In [96]:
df.comfort_convenience.value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                                                                                 216
Air conditioning,Electrica

In [97]:
df.comfort_convenience.fillna("-",inplace=True)

In [98]:
fill(df, "make_model", "body_type", "comfort_convenience", "mode")

Number of NaN :  0
------------------
-                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                                                                         

In [99]:
fill(df, "make_model", "body_type", "entertainment_media", "mode")

Number of NaN :  0
------------------
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                                        1738
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                                    1085
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                                          1081
On-board computer                                                                                                  615
Radio                                                                                                              558
Bluetooth,Hands-free equipment,On-board computer,Radio                                                             494
On-board computer,Radio                                                                                            487
Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB                                               466
Bluetooth,

In [100]:
fill(df, "make_model", "body_type", "extras", "mode")

Number of NaN :  0
------------------
Alloy wheels                                                                                                                                   5872
Alloy wheels,Touch screen                                                                                                                       697
Alloy wheels,Voice Control                                                                                                                      582
Alloy wheels,Touch screen,Voice Control                                                                                                         568
Roof rack                                                                                                                                       531
Alloy wheels,Roof rack                                                                                                                          484
Alloy wheels,Sport seats                                                  

In [101]:
fill(df, "make_model", "body_type", "safety_security", "mode")

Number of NaN :  0
------------------
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                   658
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                                                              480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                       

## gears

In [103]:
df.gears.value_counts(dropna=False)

6.000     5822
NaN       4712
5.000     3239
7.000     1908
8.000      224
9.000        6
4.000        2
1.000        2
3.000        2
50.000       1
2.000        1
Name: gears, dtype: int64

In [105]:
df.gears.replace(50,5,inplace=True)

In [106]:
df.groupby(["make_model","gears"]).hp_kW.describe()

count    mean    std     min     25%     50%     75%  \
make_model     gears                                                           
Audi A1        5.000  817.000  69.825  7.173  60.000  66.000  70.000  70.000   
               6.000  186.000  86.968 10.561  66.000  85.000  85.000  85.000   
               7.000  656.000  81.800 13.875  66.000  70.000  85.000  85.000   
               8.000    2.000  65.000  7.071  60.000  62.500  65.000  67.500   
Audi A3        5.000    2.000  81.000  0.000  81.000  81.000  81.000  81.000   
               6.000 1258.000  88.998 11.390  81.000  82.750  85.000  85.000   
               7.000 1111.000  91.115 29.379  81.000  81.000  85.000  85.000   
Opel Astra     5.000  385.000  75.094  4.682  66.000  74.000  77.000  77.000   
               6.000 1347.000  96.636 15.583  60.000  81.000 100.000 110.000   
               7.000    1.000 100.000    NaN 100.000 100.000 100.000 100.000   
Opel Corsa     1.000    1.000  66.000    NaN  66.000  66.000  66.000  66.000   
               2.000    1.000  51.000    NaN  51.000  51.000  51.000  51.000   
               5.000 1143.000  56.884  7.128  40.000  51.000  51.000  66.000   
               6.000  462.000  66.372  1.759  55.000  66.000  66.000  66.000   
               8.000    1.000  66.000    NaN  66.000  66.000  66.000  66.000   
Opel Insignia  5.000    7.000 103.571  7.700 100.000 100.500 101.000 101.000   
               6.000 1672.000 112.026 16.252  81.000 100.000 103.000 125.000   
               7.000    2.000 100.000  0.000 100.000 100.000 100.000 100.000   
               8.000  221.000 131.919 14.473 100.000 125.000 125.000 125.000   
Renault Clio   1.000    1.000 147.000    NaN 147.000 147.000 147.000 147.000   
               5.000  864.000  56.917  4.450  40.000  55.000  55.000  56.000   
               6.000  391.000  95.506 36.046  54.000  66.000  87.000  88.000   
               7.000    3.000 113.667 42.736  88.000  89.000  90.000 126.500   
Renault Duster 6.000   25.000  77.160  0.800  77.000  77.000  77.000  77.000   
Renault Espace 3.000    2.000 132.000 19.799 118.000 125.000 132.000 139.000   
               4.000    2.000 165.000  0.000 165.000 165.000 165.000 165.000   
               5.000    2.000 118.000  0.000 118.000 118.000 118.000 118.000   
               6.000  479.000 118.868 12.649  96.000 118.000 118.000 118.000   
               7.000  135.000 157.067 13.115 118.000 147.000 165.000 165.000   
               9.000    6.000 161.833  7.757 146.000 165.000 165.000 165.000   

                         max  
make_model     gears          
Audi A1        5.000  87.000  
               6.000 147.000  
               7.000 147.000  
               8.000  70.000  
Audi A3        5.000  81.000  
               6.000 162.000  
               7.000 294.000  
Opel Astra     5.000  92.000  
               6.000 149.000  
               7.000 100.000  
Opel Corsa     1.000  66.000  
               2.000  51.000  
               5.000  88.000  
               6.000  85.000  
               8.000  66.000  
Opel Insignia  5.000 121.000  
               6.000 195.000  
               7.000 100.000  
               8.000 191.000  
Renault Clio   1.000 147.000  
               5.000  88.000  
               6.000 162.000  
               7.000 163.000  
Renault Duster 6.000  81.000  
Renault Espace 3.000 146.000  
               4.000 165.000  
               5.000 118.000  
               6.000 165.000  
               7.000 165.000  
               9.000 165.000

In [116]:
df.groupby("gearing_type").gears.value_counts(dropna=False)

gearing_type    gears
Automatic       6.000    4326
                7.000    2531
                8.000     222
                5.000     207
                9.000       6
                3.000       2
                4.000       2
                1.000       1
Manual          6.000    4147
                5.000    3991
                7.000      11
                8.000       3
                2.000       1
Semi-automatic  7.000     268
                6.000     141
                5.000      59
                1.000       1
Name: gears, dtype: int64

In [112]:
df_auto = df[df.gearing_type == "Automatic"]
fill(df_auto, "make_model", "body_type", "gears", "mode")

Number of NaN :  0
------------------
6.000    4326
7.000    2531
8.000     222
5.000     207
9.000       6
4.000       2
3.000       2
1.000       1
Name: gears, dtype: int64


In [113]:
df_semi = df[df.gearing_type == "Semi-automatic"]
fill(df_semi, "make_model", "body_type", "gears", "mode")

Number of NaN :  0
------------------
7.000    268
6.000    141
5.000     59
1.000      1
Name: gears, dtype: int64


In [114]:
df_manual = df[df.gearing_type == "Manual"]
fill(df_manual, "make_model", "body_type", "gears", "mode")

Number of NaN :  0
------------------
6.000    4147
5.000    3991
7.000      11
8.000       3
2.000       1
Name: gears, dtype: int64


In [115]:
df.loc[df.gearing_type == "Manual"] = df_manual
df.loc[df.gearing_type == "Semi-automatic"] = df_semi
df.loc[df.gearing_type == "Automatic"] = df_auto

## hp_kW

In [117]:
df.hp_kW.value_counts(dropna=False)

85.000     2542
66.000     2122
81.000     1402
100.000    1308
110.000    1112
70.000      888
125.000     707
51.000      695
55.000      569
118.000     516
92.000      466
121.000     392
147.000     380
77.000      345
56.000      286
54.000      276
103.000     253
87.000      232
165.000     194
88.000      177
60.000      160
162.000      98
NaN          88
74.000       81
96.000       72
71.000       59
101.000      47
67.000       40
154.000      39
122.000      35
119.000      30
164.000      27
135.000      24
82.000       22
52.000       22
1.000        20
78.000       20
146.000      18
294.000      18
141.000      16
57.000       10
120.000       8
104.000       8
191.000       7
112.000       7
155.000       6
117.000       6
184.000       5
90.000        4
76.000        4
65.000        4
149.000       3
98.000        3
93.000        3
80.000        3
168.000       3
150.000       2
63.000        2
140.000       2
86.000        2
89.000        2
40.000        2
167.000 

In [118]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

Number of NaN :  0
------------------
85.000     2543
66.000     2125
81.000     1403
100.000    1315
110.000    1113
70.000      890
125.000     710
51.000      695
55.000      589
118.000     550
92.000      466
121.000     392
147.000     380
77.000      353
56.000      294
54.000      276
103.000     253
87.000      232
165.000     194
88.000      177
60.000      160
162.000      98
74.000       81
96.000       72
71.000       59
101.000      47
67.000       40
154.000      39
122.000      35
119.000      30
164.000      27
135.000      24
52.000       22
82.000       22
78.000       20
1.000        20
146.000      18
294.000      18
141.000      16
57.000       10
120.000       8
104.000       8
191.000       7
112.000       7
155.000       6
117.000       6
184.000       5
76.000        4
65.000        4
90.000        4
80.000        3
98.000        3
168.000       3
93.000        3
149.000       3
150.000       2
89.000        2
63.000        2
86.000        2
53.000        2
27

# upholstery_type

In [119]:
df.upholstery_type.value_counts(dropna=False)

NaN             14617
Cloth            1005
Part leather      140
Full leather      139
Velour             16
alcantara           2
Name: upholstery_type, dtype: int64

In [ ]:
# too much NaN values. It probably affects the price but I would rather dropping the column.

In [121]:
df.groupby("make_model")["upholstery_type"].value_counts(dropna=False)

make_model      upholstery_type
Audi A1         NaN                2530
                Cloth                69
                Velour                9
                Part leather          4
                Full leather          2
Audi A2         NaN                   1
Audi A3         NaN                2883
                Cloth               177
                Part leather         18
                Full leather         14
                Velour                4
                alcantara             1
Opel Astra      NaN                2260
                Cloth               239
                Part leather         15
                Full leather         11
                alcantara             1
Opel Corsa      NaN                2000
                Cloth               216
                Velour                3
Opel Insignia   NaN                2392
                Cloth               115
                Full leather         54
                Part leather         37
Renault 

In [122]:
df.drop("upholstery_type", axis=1, inplace=True)

## warranty month

In [123]:
df.warranty_months.value_counts(dropna=False)

NaN       11066
12.000     2594
24.000     1118
60.000      401
36.000      279
48.000      149
6.000       125
72.000       59
3.000        33
23.000       11
18.000       10
20.000        7
25.000        6
2.000         5
50.000        4
26.000        4
16.000        4
19.000        3
1.000         3
4.000         3
13.000        3
34.000        3
45.000        2
14.000        2
17.000        2
11.000        2
46.000        2
28.000        2
21.000        2
22.000        2
9.000         2
30.000        1
33.000        1
56.000        1
40.000        1
7.000         1
15.000        1
8.000         1
10.000        1
49.000        1
47.000        1
65.000        1
Name: warranty_months, dtype: int64

In [128]:
df.groupby(["make_model","age"])["warranty_months"].describe()

count   mean    std    min    25%    50%    75%    max
make_model     age                                                           
Audi A1        0.000 205.000 27.990 13.196 12.000 24.000 24.000 36.000 60.000
               1.000 234.000 26.291 14.306  6.000 12.000 24.000 36.000 60.000
               2.000 131.000 17.679  9.959  3.000 12.000 12.000 24.000 60.000
               3.000 149.000 13.926  8.677  3.000 12.000 12.000 12.000 60.000
Audi A2        1.000   0.000    NaN    NaN    NaN    NaN    NaN    NaN    NaN
Audi A3        0.000 251.000 27.697 11.454  3.000 24.000 24.000 36.000 72.000
               1.000 290.000 21.707 11.066  3.000 12.000 24.000 24.000 60.000
               2.000 223.000 16.655  8.650  6.000 12.000 12.000 24.000 60.000
               3.000 278.000 17.112  9.984  3.000 12.000 12.000 12.000 36.000
Opel Astra     0.000 168.000 37.804 23.263  2.000 12.000 24.000 60.000 72.000
               1.000 220.000 25.795 19.511  6.000 12.000 12.000 24.000 60.000
               2.000 166.000 14.988 10.616  6.000 12.000 12.000 12.000 60.000
               3.000 158.000 24.797 21.027  6.000 12.000 12.000 54.000 60.000
Opel Corsa     0.000 167.000 27.389 15.156  3.000 24.000 24.000 24.000 72.000
               1.000 215.000 17.386 11.061  3.000 12.000 12.000 24.000 60.000
               2.000 151.000 22.172 18.919  1.000 12.000 12.000 24.000 60.000
               3.000 148.000 27.061 22.279  3.000 12.000 12.000 60.000 60.000
Opel Insignia  0.000 183.000 31.885 19.836  3.000 24.000 24.000 24.000 72.000
               1.000 207.000 25.290 20.444  1.000 12.000 12.000 36.000 60.000
               2.000 206.000 17.024 11.912  2.000 12.000 12.000 12.000 60.000
               3.000 173.000 15.225 11.565  2.000 12.000 12.000 12.000 60.000
Renault Clio   0.000  98.000 35.510 15.049 12.000 24.000 24.000 48.000 65.000
               1.000 118.000 15.686  8.762  6.000 12.000 12.000 12.750 48.000
               2.000 180.000 16.733 11.407  3.000 12.000 12.000 24.000 60.000
               3.000 162.000 10.864  3.772  3.000 12.000 12.000 12.000 36.000
Renault Duster 0.000  32.000 12.000  0.000 12.000 12.000 12.000 12.000 12.000
Renault Espace 0.000  55.000 37.618 17.231 12.000 24.000 36.000 54.000 60.000
               1.000  85.000 22.400 13.237 12.000 12.000 21.000 25.000 60.000
               2.000  61.000 14.934  9.882  3.000 12.000 12.000 12.000 60.000
               3.000 139.000 12.482  6.410  1.000 12.000 12.000 12.000 60.000

In [129]:
fill(df, "make_model", "age", "warranty_months", "median")

Number of NaN :  0
------------------
12.000    9220
24.000    5177
36.000     494
60.000     401
21.000     168
48.000     149
6.000      125
72.000      59
3.000       33
23.000      11
18.000      10
20.000       7
25.000       6
2.000        5
16.000       4
26.000       4
50.000       4
4.000        3
34.000       3
13.000       3
19.000       3
1.000        3
45.000       2
11.000       2
46.000       2
9.000        2
14.000       2
28.000       2
17.000       2
22.000       2
15.000       1
33.000       1
30.000       1
56.000       1
7.000        1
8.000        1
10.000       1
49.000       1
47.000       1
40.000       1
65.000       1
Name: warranty_months, dtype: int64


## displacement_cc

In [131]:
df["displacement_cc"].value_counts(dropna=False)

1598.000     4761
999.000      2438
1398.000     1314
1399.000      749
1229.000      677
1956.000      670
1461.000      595
1490.000      559
NaN           496
1422.000      467
1197.000      353
898.000       351
1395.000      320
1968.000      301
1149.000      288
1618.000      212
1798.000      210
1498.000      196
1600.000      130
1248.000      110
1997.000      103
1364.000      102
1400.000       90
998.000        72
1500.000       50
2000.000       46
1000.000       40
1.000          36
1998.000       25
2480.000       20
1984.000       18
1200.000       18
899.000        11
1397.000       11
160.000         6
1499.000        5
929.000         5
139.000         4
900.000         4
1596.000        4
997.000         4
1199.000        3
1599.000        3
1396.000        3
1495.000        2
1300.000        2
1589.000        2
2.000           2
995.000         2
1568.000        1
1584.000        1
16000.000       1
1369.000        1
996.000         1
1686.000        1
1100.000  

In [132]:
df.groupby("hp_kW").displacement_cc.describe()

,count,mean,std,min,25%,50%,75%,max
hp_kW,,,,,,,,
1.000,20.000,1361.150,201.358,998.000,1229.000,1364.000,1591.250,1598.000
4.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40.000,2.000,1217.000,345.068,973.000,1095.000,1217.000,1339.000,1461.000
44.000,1.000,1364.000,NaN,1364.000,1364.000,1364.000,1364.000,1364.000
51.000,664.000,1226.750,44.528,122.000,1229.000,1229.000,1229.000,1398.000
52.000,22.000,1229.000,0.000,1229.000,1229.000,1229.000,1229.000,1229.000
53.000,1.000,1149.000,NaN,1149.000,1149.000,1149.000,1149.000,1149.000
54.000,274.000,1089.073,253.067,1.000,1149.000,1149.000,1149.000,1200.000


In [134]:
df.groupby("make_model")["displacement_cc"].describe()

,count,mean,std,min,25%,50%,75%,max
make_model,,,,,,,,
Audi A1,2585.000,1188.747,247.410,929.000,999.000,999.000,1422.000,1984.000
Audi A2,1.000,1598.000,NaN,1598.000,1598.000,1598.000,1598.000,1598.000
Audi A3,3047.000,1510.704,285.370,997.000,1498.000,1598.000,1598.000,2480.000
Opel Astra,2437.000,1428.114,367.625,139.000,1399.000,1399.000,1598.000,15898.000
Opel Corsa,2145.000,1332.831,87.794,122.000,1229.000,1398.000,1398.000,1400.000
Opel Insignia,2523.000,1662.636,380.984,1.000,1490.000,1598.000,1956.000,16000.000
Renault Clio,1748.000,1241.514,260.937,1.000,1149.000,1197.000,1461.000,1618.000
Renault Duster,33.000,1695.030,174.043,1598.000,1598.000,1598.000,1600.000,1998.000
Renault Espace,904.000,1678.627,184.803,160.000,1598.000,1598.000,1798.000,2000.000


In [135]:
fill(df, "make_model", "hp_kW", "displacement_cc", "median")

Number of NaN :  0
------------------
1598.000     4986
999.000      2466
1398.000     1349
1399.000      769
1229.000      708
1956.000      696
1461.000      662
1490.000      570
1422.000      472
898.000       367
1197.000      357
1395.000      325
1968.000      304
1149.000      291
1618.000      215
1798.000      214
1498.000      196
1600.000      130
1248.000      117
1364.000      104
1997.000      103
1400.000       90
998.000        72
1500.000       50
2000.000       46
1000.000       40
1.000          36
1998.000       26
2480.000       20
1984.000       18
1200.000       18
1397.000       11
899.000        11
160.000         6
929.000         5
1499.000        5
139.000         4
1596.000        4
997.000         4
900.000         4
1396.000        3
1599.000        3
1199.000        3
1589.000        2
2.000           2
995.000         2
1300.000        2
1495.000        2
15898.000       1
1568.000        1
1368.000        1
890.000         1
1239.000        1
1496.000

## weight_kg

In [136]:
df.weight_kg.value_counts(dropna=False)

NaN         6974
1163.000     574
1360.000     356
1165.000     301
1335.000     242
1135.000     213
1199.000     205
1734.000     170
1180.000     168
1503.000     165
1350.000     155
1355.000     135
1260.000     127
1275.000     112
1278.000     110
1425.000     109
1487.000     109
1255.000     108
1200.000     107
1522.000     103
1273.000     103
1280.000     102
1403.000      91
1120.000      90
1195.000      89
1659.000      89
1701.000      87
1250.000      84
1441.000      82
1308.000      80
1285.000      80
1110.000      75
1613.000      75
1279.000      72
1364.000      70
1345.000      67
1733.000      65
1685.000      64
1071.000      64
1325.000      64
1141.000      64
1230.000      63
1845.000      56
1090.000      54
1052.000      53
1664.000      52
1154.000      52
1513.000      51
1065.000      50
1237.000      49
1440.000      46
1205.000      46
1119.000      46
1088.000      46
1265.000      45
1395.000      44
1666.000      43
1585.000      43
1209.000      

In [137]:
df.groupby(["make_model","hp_kW"]).weight_kg.describe()

count     mean     std      min      25%      50%  \
make_model     hp_kW                                                         
Audi A1        60.000   74.000 1179.635 156.463 1088.000 1110.000 1135.000   
               66.000  240.000 1227.229 119.162 1105.000 1195.000 1195.000   
               67.000    3.000 1195.000   0.000 1195.000 1195.000 1195.000   
               70.000  454.000 1185.742 155.516  102.000 1135.000 1135.000   
               71.000   35.000 1071.857  25.237 1035.000 1065.000 1065.000   
               85.000  545.000 1281.356 167.942 1100.000 1180.000 1225.000   
               86.000    0.000      NaN     NaN      NaN      NaN      NaN   
               87.000    3.000 1250.000   0.000 1250.000 1250.000 1250.000   
               92.000   66.000 1222.576 123.762 1115.000 1180.000 1180.000   
               93.000    1.000 1115.000     NaN 1115.000 1115.000 1115.000   
               110.000  19.000 1240.263 103.487 1180.000 1192.500 1230.000   
               112.000   0.000      NaN     NaN      NaN      NaN      NaN   
               140.000   1.000 1205.000     NaN 1205.000 1205.000 1205.000   
               141.000   8.000 1323.750 154.544 1255.000 1255.000 1280.000   
               147.000   8.000 1284.375  72.281 1180.000 1225.000 1335.000   
Audi A2        85.000    0.000      NaN     NaN      NaN      NaN      NaN   
Audi A3        9.000     0.000      NaN     NaN      NaN      NaN      NaN   
               81.000  522.000 1381.239 150.602 1180.000 1335.000 1355.000   
               82.000   10.000 1279.000  60.544 1180.000 1255.000 1255.000   
               85.000  976.000 1386.002 191.531 1155.000 1265.000 1335.000   
               86.000    0.000      NaN     NaN      NaN      NaN      NaN   
               87.000    3.000 1360.000   0.000 1360.000 1360.000 1360.000   
               92.000   50.000 1290.800  69.857 1205.000 1280.000 1280.000   
               96.000    8.000 1330.625  58.152 1275.000 1275.000 1332.500   
               100.000   1.000 1390.000     NaN 1390.000 1390.000 1390.000   
               110.000 297.000 1383.202 125.547 1041.000 1325.000 1379.000   
               112.000   0.000      NaN     NaN      NaN      NaN      NaN   
               132.000   0.000      NaN     NaN      NaN      NaN      NaN   
               133.000   1.000 1380.000     NaN 1380.000 1380.000 1380.000   
               135.000  19.000 1521.316 107.197 1395.000 1500.000 1500.000   
               137.000   0.000      NaN     NaN      NaN      NaN      NaN   
               140.000   1.000 1490.000     NaN 1490.000 1490.000 1490.000   
               150.000   0.000      NaN     NaN      NaN      NaN      NaN   
               162.000   1.000 1470.000     NaN 1470.000 1470.000 1470.000   
               228.000   1.000 1635.000     NaN 1635.000 1635.000 1635.000   
               270.000   2.000 1595.000   0.000 1595.000 1595.000 1595.000   
               294.000  18.000 1574.722  31.784 1485.000 1585.000 1585.000   
Opel Astra     1.000     0.000      NaN     NaN      NaN      NaN      NaN   
               60.000    1.000 1360.000     NaN 1360.000 1360.000 1360.000   
               66.000   28.000 1276.214   9.739 1263.000 1273.000 1273.000   
               67.000    0.000      NaN     NaN      NaN      NaN      NaN   
               70.000   15.000 1357.600  37.496 1270.000 1345.000 1364.000   
               74.000   13.000 1259.615  49.755 1188.000 1244.000 1244.000   
               76.000    4.000 1166.750   7.500 1163.000 1163.000 1163.000   
               77.000  156.000 1262.660  37.634 1163.000 1263.000 1273.000   
               78.000   10.000 1364.000   0.000 1364.000 1364.000 1364.000   
               80.000    0.000      NaN     NaN      NaN      NaN      NaN   
               81.000  375.000 1406.491 115.468 1235.000 1350.000 1364.000   
               82.000    6.000 1294.500  58.821 1235.000 1260.750 1293.000   
               84.000    1.000 1549.000     NaN

In [138]:
fill(df, "make_model", "hp_kW", "weight_kg", "median")

Number of NaN :  0
------------------
1163.000    1504
1335.000     908
1135.000     634
1165.000     544
1308.000     457
1666.000     451
1487.000     438
1734.000     437
1355.000     398
1225.000     364
1360.000     362
1195.000     325
1564.000     324
1273.000     317
1364.000     314
1180.000     274
1406.500     224
1278.000     211
1379.000     207
1199.000     205
1503.000     194
1059.000     156
1350.000     155
1280.000     153
1146.000     149
1685.000     137
1260.000     127
1552.000     121
1255.000     113
1275.000     112
1425.000     109
1200.000     107
1162.000     107
1522.000     103
1090.000      94
1345.000      92
1403.000      91
1279.000      91
1120.000      90
1659.000      89
1701.000      87
1250.000      85
1441.000      82
1285.000      80
1237.000      76
1110.000      75
1613.000      75
1230.000      74
1196.000      73
1065.000      71
1758.000      68
1733.000      68
1088.000      66
1071.000      64
1141.000      64
1325.000      64
1244.000  

## cons_comb & cons_city & cons_country
I will use only cons_comb. but I can use others to fill the missing values.

In [140]:
df[df.cons_comb.isnull()]["cons_city"].value_counts(dropna=False)

NaN      1925
3.400      37
3.500      19
3.300      19
4.600      19
5.000       6
4.500       3
6.700       2
5.200       1
7.100       1
4.000       1
Name: cons_city, dtype: int64

In [141]:
df[df.cons_comb.isnull()]["cons_country"].value_counts(dropna=False)

NaN      1906
3.800      56
5.700      32
3.500      19
4.400       5
5.100       4
3.400       3
4.200       2
3.900       1
5.800       1
4.500       1
3.300       1
5.600       1
4.000       1
Name: cons_country, dtype: int64

In [144]:
df.cons_comb.fillna(df.groupby("cons_city").cons_comb.transform("median"), inplace=True)

In [145]:
df[df.cons_comb.isnull()]["cons_country"].value_counts(dropna=False)

NaN      1906
5.700      13
5.100       4
5.800       1
5.600       1
Name: cons_country, dtype: int64

In [146]:
df.cons_comb.fillna(df.groupby("cons_country").cons_comb.transform("median"), inplace=True)

In [147]:
df.cons_comb.value_counts(dropna=False)

NaN       1906
5.400      773
3.900      736
4.000      732
5.100      657
4.400      624
5.600      618
4.700      602
3.800      585
4.800      546
5.000      545
4.500      523
5.200      454
4.200      441
4.600      426
4.900      393
5.300      380
5.500      380
5.900      373
3.700      369
5.700      342
4.100      342
6.000      331
3.300      326
4.300      307
3.500      289
6.200      216
3.600      194
6.300      181
6.100      175
5.800      164
6.800      149
6.600      149
3.400      106
3.200       81
6.400       75
3.000       69
7.400       66
6.700       43
6.500       43
7.100       39
10.000      34
6.900       27
8.300       20
7.600       14
7.000       10
3.100        7
7.200        6
7.800        6
8.000        5
51.000       4
8.600        4
8.700        3
1.600        3
7.900        3
38.000       2
0.000        2
40.000       2
7.300        2
8.100        2
32.000       1
1.200        1
33.000       1
50.000       1
11.000       1
55.000       1
54.000    

In [148]:
df.groupby(["make_model", "hp_kW"]).cons_comb.describe()

count   mean   std    min    25%    50%    75%  \
make_model     hp_kW                                                       
Audi A1        60.000   143.000  4.265 0.116  3.900  4.200  4.200  4.400   
               66.000   436.000  3.839 0.185  3.000  3.700  3.900  4.000   
               67.000     3.000  3.700 0.000  3.700  3.700  3.700  3.700   
               70.000   746.000  4.708 3.705  4.000  4.200  4.400  4.500   
               71.000    32.000  4.347 0.084  4.200  4.300  4.400  4.400   
               85.000   847.000  4.453 0.526  3.000  3.900  4.800  4.800   
               86.000     0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               87.000     3.000  3.700 0.000  3.700  3.700  3.700  3.700   
               92.000   161.000  5.027 0.145  4.800  4.900  5.000  5.100   
               93.000     1.000  4.900   NaN  4.900  4.900  4.900  4.900   
               110.000   30.000  4.900 0.182  4.700  4.700  5.000  5.100   
               112.000    0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               140.000    1.000  5.800   NaN  5.800  5.800  5.800  5.800   
               141.000   14.000  5.643 0.085  5.600  5.600  5.600  5.600   
               147.000   13.000  6.000 0.000  6.000  6.000  6.000  6.000   
Audi A2        85.000     1.000  4.900   NaN  4.900  4.900  4.900  4.900   
Audi A3        9.000      0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               81.000   697.000  3.956 1.877  3.000  3.800  3.800  4.000   
               82.000    10.000  3.830 0.157  3.400  3.825  3.900  3.900   
               85.000  1515.000  4.377 1.375  3.500  4.000  4.100  4.700   
               86.000     1.000  3.000   NaN  3.000  3.000  3.000  3.000   
               87.000     3.000  4.000 0.000  4.000  4.000  4.000  4.000   
               92.000    63.000  5.090 0.100  4.900  5.100  5.100  5.100   
               96.000     8.000  4.938 1.595  3.500  3.500  5.100  5.175   
               100.000    1.000  4.000   NaN  4.000  4.000  4.000  4.000   
               110.000  465.000  4.688 0.521  1.600  4.400  4.500  5.100   
               112.000    0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               132.000    1.000  5.800   NaN  5.800  5.800  5.800  5.800   
               133.000    1.000  4.300   NaN  4.300  4.300  4.300  4.300   
               135.000   23.000  4.709 0.294  4.100  4.800  4.800  4.900   
               137.000    0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               140.000    1.000  6.400   NaN  6.400  6.400  6.400  6.400   
               150.000    0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               162.000    1.000  6.600   NaN  6.600  6.600  6.600  6.600   
               228.000    2.000  6.800 0.424  6.500  6.650  6.800  6.950   
               270.000    2.000  8.100 0.000  8.100  8.100  8.100  8.100   
               294.000   18.000  8.300 0.000  8.300  8.300  8.300  8.300   
Opel Astra     1.000      0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               60.000     4.000  3.000 0.000  3.000  3.000  3.000  3.000   
               66.000    64.000  4.812 0.281  4.300  4.700  4.700  5.000   
               67.000     0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               70.000    36.000  3.622 0.054  3.600  3.600  3.600  3.600   
               74.000    60.000  5.442 0.118  5.400  5.400  5.400  5.400   
               76.000     4.000  4.400 0.141  4.200  4.350  4.450  4.500   
               77.000   237.000  4.505 0.280  3.900  4.300  4.500  4.700   
               78.000    13.000  4.469 0.118  4.400  4.400  4.400  4.500   
               80.000     0.000    NaN   NaN    NaN    NaN    NaN    NaN   
               81.000   490.000  3.624 0.287  3.000  3.500  3.600  3.700   
               82.000     6.000  3.500 0.110  3.400  3.425  3.500  3.500   
               84.000     1.000  6.300   NaN  6.300  6.300  6.300  6.300   
               85.000    19.000  6.558 0.340  6.200  6.300  6.400  6.650  

In [149]:
fill(df, "make_model", "hp_kW", "cons_comb", "median")

Number of NaN :  0
------------------
5.400     908
4.500     888
3.900     787
5.600     764
4.700     764
4.000     738
5.100     731
4.400     710
3.800     691
5.000     655
4.800     590
4.100     461
5.200     454
4.200     454
5.900     453
5.500     447
4.600     426
4.900     395
5.300     382
3.300     380
3.700     379
5.700     370
3.500     334
6.000     333
4.300     329
3.600     292
6.200     216
6.100     214
6.300     181
5.800     164
6.800     162
6.600     153
3.400     106
6.400      92
3.200      81
3.000      69
7.400      66
6.500      43
6.700      43
7.100      39
10.000     35
6.900      27
8.300      20
7.600      14
7.000      10
3.100       7
7.200       6
7.800       6
8.600       5
8.000       5
5.150       4
51.000      4
7.900       3
8.700       3
1.600       3
0.000       2
8.100       2
38.000      2
40.000      2
7.300       2
46.000      1
54.000      1
13.800      1
11.000      1
32.000      1
33.000      1
50.000      1
43.000      1
1.200     

In [150]:
df.drop(["cons_city","cons_country"],axis=1, inplace=True)

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make_model           15919 non-null  object 
 1   body_type            15919 non-null  object 
 2   price                15919 non-null  int64  
 3   km                   15919 non-null  float64
 4   type                 15919 non-null  object 
 5   previous_owners      15919 non-null  float64
 6   inspection_new       15919 non-null  int64  
 7   make                 15919 non-null  object 
 8   model                15919 non-null  object 
 9   paint_type           15919 non-null  object 
 10  nr_of_doors          15919 non-null  float64
 11  nr_of_seats          15919 non-null  float64
 12  gearing_type         15919 non-null  object 
 13  drive_chain          15919 non-null  object 
 14  fuel                 15919 non-null  object 
 15  co2_emission         15919 non-null 

In [152]:
df.to_csv("filled_scout.csv", index=False)